# JPE cleaning
This notebook walks through how the JPE articles were sorted into categories of articles and non-articles.

## Loading libraries

In [1]:
from tokenize import Ignore
from numpy import NaN
import pandas as pd
from difflib import SequenceMatcher
import multiprocessing as mp
import time
from os import path
import os
from pathlib import Path
from PyPDF2 import PdfFileReader, PdfFileWriter
import re


## Loading Files
Please replace file paths with local file paths and comment out unapplicable content eg: datadump

In [2]:
masters = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\Master lists\\JPE_master.xlsx")
pivots = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\pivots\\JPE_pivots.xlsx")
scopus = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\Scopus\\JPE_SCOPUS.xlsx")
datadump = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_datadump.xlsx")

pd.set_option('display.max_colwidth', None)

## Create File names
Again, replace these with local file paths

In [ ]:
authors="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_authors.xlsx"
non_auth="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_Nauthors.xlsx"
saveas="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_processed.xlsx"
reviews="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_reviews.xlsx"
misc="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_misc.xlsx"
conf="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_conf.xlsx"

## Some random checks on the masters list
My assumption is that all data without author names must be miscellaneous documents like reports by the committee, forewords, front matters etc.. The goal of this notebook is to check for certain that all the documents without author names are actually miscellaneous documents and then classify them as miscellaneous (MISC). Hence, first we group everything the data by title to see the repetitive general content that can likely be removed.

In [ ]:
pd.set_option('display.max_rows',masters.shape[0])
temp=masters['title'].str.lower().value_counts()
pd.DataFrame(temp[temp>1])

Some repetitions are due to multiple comments. Now consider this list in absence of author names.

In [ ]:
temp2=masters[masters['authors'].isna()]['title'].str.lower().value_counts()
pd.DataFrame(temp2)

## Classifying miscellaneous documents

In [ ]:
scopus.rename(columns = {'abstract':'abstract2', 'title':'title2', 'authors':'authors2'}, inplace = True)
scopus['pages2']=scopus['pages']
masters['pages']=masters['pages'].str.strip()
masters.loc[masters.title.str.lower() == "back matter", 'pages'] = NaN

masters.loc[masters.apply(lambda k: SequenceMatcher(None, k['title'].lower(), 'front matter').ratio(), axis=1)>0.75,"content_type"]='MISC'
masters.loc[masters.apply(lambda k: SequenceMatcher(None, k['title'].lower(), 'back matter').ratio(), axis=1)>0.75,"content_type"]='MISC'
masters.loc[masters.apply(lambda k: SequenceMatcher(None, k['title'].lower(), 'volume information').ratio(), axis=1)>0.75,"content_type"]='MISC'
masters.loc[masters.apply(lambda k: SequenceMatcher(None, k['title'].lower(), 'books recieved').ratio(), axis=1)>0.75,"content_type"]='MISC'
masters.loc[masters.apply(lambda k: SequenceMatcher(None, k['title'].lower(), 'washington notes').ratio(), axis=1)>0.75,"content_type"]='MISC'
masters.loc[masters['title'].str.lower().str.match(r'(in )?memori(a|u)(m|l)')==True, 'content_type']='MISC'
masters.loc[masters.apply(lambda k: SequenceMatcher(None, k['title'].lower(), 'books reccieved').ratio(), axis=1)>0.75,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^journal of political economy(.*)')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^index to volume(.*)')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^new publications')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^(prefatory |\[)note(|s)(|\])$')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^(|\[)questions and answers(\]|)$')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^(|short )notice(|s)$')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^back cover(.*)')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^introduction(.*)')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^combined references(.*)')==True,'content_type']='MISC'

## Classifying other content types

In [ ]:
# check for how many articles are still unclassified
sum(masters.content_type.isna())
#masters.shape[0]

In [ ]:
masters.loc[masters['authors'].str.lower().str.match(r'^review(ed|) by(.*)')==True,'content_type']='Review' #reviews
masters.loc[(masters['title'].str.lower().str.match(r'(.*) by (.*)')==True) & (masters.authors.isna()==True),'content_type']='Review2' 
#possible reviews that don't have author names


In [ ]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*: (|a )comment(|.*)$')==True,'content_type']='Comment'
masters[masters['content_type']=='Comment'].shape[0] #comments

In [ ]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?) (|a )reply(| to.*)$')==True,'content_type']="Reply"
masters[masters['content_type']=='Reply'].shape[0]

In [ ]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?) (|a )rejoinder.*$')==True,'content_type']="Rejoinder"
masters[masters['content_type']=='Rejoinder'].shape[0]

In [ ]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*: (|a )discussion$')==True,'content_type']="Discussion"
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'(^|a )discussion(|.*)$')==True,'content_type']="Discussion"
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*:.*(|a )discussion(|s)$')==True,'content_type']='Discussion'
masters[masters['content_type']=='Discussion'].shape[0]

In [ ]:
masters.loc[masters['content_type'].isna(),'content_type']="Article"
masters[masters['content_type']=='Article'].shape[0]

In [ ]:
# block for testing regex matches
#masters[masters['title'].str.lower().str.match(r'^\washington notes$')==True]
#masters[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?) (|a )rejoinder.*$')==True]
#masters[masters.content_type=='Discussion'].shape[0]

## Consider the pivots file
At times, conference papers are structured differently to normal articles. Hence, it may be necessary to distinguish conference papers from common articles. Separate special issues (S) from normal issues (N)

In [ ]:
pivots.loc[pivots.Jstor_issue_text.str.lower().str.match(r'(.*)(supplement|proceedings|annual meeting|survey)(.*)'),'type']="S"
pivots.loc[pivots.type.isna(),'type']='N'
pivots.type.value_counts()
pivots[pivots.type=="S"]

Merge pivots and masters together

In [ ]:
result = pd.merge(masters, pivots[['issue_url','year','volume','issue','journal','type']], how="left", on=["issue_url", "issue_url"])

## Summaries of content 

In [ ]:
pd.DataFrame(result.content_type.value_counts())

In [ ]:
pd.DataFrame(result[result.year>1939].content_type.value_counts())

In [ ]:
pd.DataFrame(result[(result.year>1939) & (result.year<2011)].content_type.value_counts())

In [ ]:
result.to_excel(saveas, index=False)

## This section aims to match up Scopus records and Jstor articles
If an article's affiliations, citations or abstracts are recorded on Scopus. matching up the Scopus data is useful for comparing the textual accuracy of OCR parsers. I use volume, issue, year and page numbers which are common to both the scopus data and the Jstor metadata to match articles. 

Then I use a sequence comparison between the journal titles of the matched articles to decide if the scopus data has been matched correctly. If the match ratio is below 70%, the title is investigated and if wrong, the scopus data for that matched article is either corrected or discarded. 

The next section reads in the processed data previously saved. Please make sure the path in the variable 'cleaned' matches the file path on your local machine.

In [69]:
cleaned=pd.read_excel('C:\\Users\\sjwu1\\Journal_Data\\datadumps\\processed\\JPE_processed.xlsx')
cleaned['volume']=cleaned['volume'].astype(str)
scopus['pages']=scopus['pages'].str.strip()
#print(scopus.shape)

This is the number of entries on Scopus

In [70]:
#Note that we only have data up to 2016 in the masterlists because of the moving wall on JSTOR
print(sum(scopus['year']<2016))

965


Merge on year, issue, volume and pages.

In [71]:
Merged=pd.merge(cleaned, scopus, on=['year', 'issue','volume','pages'], how='left')
pd.set_option('display.max_columns', None)

Check how many of the scopus entries matched. Good news! Only 9 did not match

In [72]:
sum(Merged['title_y'].isna()==False)

956

Consider the titles that have a less than 70% match.

In [73]:
count=0
for m in Merged.index:
    
    if(pd.isna(Merged.iloc[m]['title_y'])==False):
        ratio=SequenceMatcher(None, Merged.iloc[m]['title_x'].lower(), Merged.iloc[m]['title_y'].lower()).ratio()

        if((ratio<0.7) & (Merged.iloc[m]['content_type']!='MISC')):
            print(Merged.iloc[m]['year'])
            print(ratio)
            count+=1
            A_ratio=SequenceMatcher(None, Merged.iloc[m]['authors_x'].lower(), Merged.iloc[m]['authors_y'].lower()).ratio()
            print(A_ratio)
            print(Merged.iloc[m]['stable_url'])
            print('vol: '+str(Merged.iloc[m]['volume']))
            print('issue: '+str(Merged.iloc[m]['issue']))
            print('pages: '+Merged.iloc[m]['pages'])
            print('jstor: '+Merged.iloc[m]['title_x'])
            print('scopus: '+Merged.iloc[m]['title_y'])
            print('jstor: '+Merged.iloc[m]['authors_x'])
            print('scopus: '+Merged.iloc[m]['authors_y'])
            print('scopus index: '+str(scopus[scopus['title']==Merged.iloc[m]['title_y']].index))
            print(m)
            print()
print(count)

2016
0.25757575757575757
0.18181818181818182
https://www.jstor.org/stable/26549907
vol: 124
issue: 5
pages: 1466-1514
jstor: Thomas Jefferson on the “Public Good” Nature of Knowledge
scopus: Downward nominal wage rigidity, currency pegs, and involuntary unemployment
jstor: Marcos Dal Bianco
scopus: Schmitt-Grohé, S.--a--
Uribe, M.--b-- 
scopus index: Int64Index([1014], dtype='int64')
20

2016
0.2857142857142857
0.23684210526315788
https://www.jstor.org/stable/26549896
vol: 124
issue: 4
pages: 1187-1234
jstor: “Market for Lemons” Applied to Social Groups
scopus: Intermittency and the value of renewable energy
jstor: Joshua S. Goodman
scopus: Gowrisankaran, G.--a--
Reynolds, S.S.--b--
Samano, M.--c-- 
scopus index: Int64Index([1021], dtype='int64')
30

2016
0.24390243902439024
0.17391304347826086
https://www.jstor.org/stable/26549885
vol: 124
issue: 3
pages: 826-878
jstor: The Gambler’s Fallacy in Existentialist Tragicomedy
scopus: Capabilities, wealth, and trade
jstor: James Marrone
sco

Manually correct some errors in both data sets

In [74]:
cleaned.at[20, 'pages']=float('nan')
cleaned.at[30, 'pages']=float('nan')
cleaned.at[39, 'pages']=float('nan')
cleaned.at[48, 'pages']=float('nan')
scopus.at[633, 'title']='Erratum: The accident externality from driving'

Re-merge the scopus data and the cleaned masterlists

In [75]:
Merged=pd.merge(cleaned, scopus, on=['year', 'issue','volume','pages'], how='left')

Check for whether there is an issue in document type

In [76]:
pd.DataFrame(Merged[Merged['document type'].str.len()>100])

,stable_url,authors_x,title_x,abstract_x,content_type,issue_url,pages,year,volume,issue,journal_x,type,authors_y,title_y,journal_y,DOI,affiliations,abstract_y,citations,document type,index keywords,author keywords


## Merge datadump.xlsx

Datadump.xlsx contains reference metadata scraped from JSTOR during sessions running Stage_2_scraper.py. This section merges both the scopus and cleaned data with the references from datadumps. Then this is saved as an excel file. replace path as necessary.

In [77]:
Merged=pd.merge(Merged,datadump[['stable_url', 'footnotes','raw','references']], on=['stable_url'], how='left')

In [ ]:
Merged.rename(columns = {'authors_x':'Jstor_authors', 'title_x':'Jstor_title', 'abstract_x':'Jstor_abstract', 'journal_x':'Jstor_journal', 'authors_y':'scopus_authors','title_y':'scopus_title', 'abstract_y':'scopus_abstract', 'journal_y':'scopus_journal' }, inplace = True)

Saving

In [98]:
Merged.to_excel('C:\\Users\\sjwu1\\Journal_Data\\datadumps\\JPE_M_sco_du.xlsx', index=False)

# Merged dataset field description

This is a description of fields in the Merged dataset that combines JPE masterlist, pivot list, Scopus data pre-2016. The Merged dataset is stored in JPE_M_sco_du.xlsx.

    'stable_url' : JSTOR url for article 
    'Jstor_authors' : Author names recorded by JSTOR
    'Jstor_title' : Title of article recorded by JSTOR
    'Jstor_abstract' : abstract recorded by JSTOR nb: this is blank at the moment
    'content_type' : Article type determined during cleaning. Includes MISC for miscellaneous, Reviews, Note, Comment, Rejoinder and Article categorizations
    'issue_url' : url of issue article belongs to on JSTOR
    'pages' : pages as recorded by JSTOR
    'year' : Year of publication recorded by JSTOR
    'volume' : Volume of article recorded by JSTOR
    'issue' : issue of article recorded by JSTOR
    'Jstor_journal' : journal name JSTOR
    'type' : Type of issue determined during cleaning. S for special issue. N for normal issue
    'scopus_authors' : Author names recorded by Scopus
    'scopus_title' : Title recorded by Scopus
    'scopus_journal' : Journal name recorded by Scopus
    'DOI' : DOI recorded by scopus
    'affiliations' : affiliations of authors as recorded by scopus
    'scopus_abstract' : abstract of article recorded by scopus
    'citations' : citations of article recorded by scopus
    'document type' : Article type recorded by scopus, may differ from that in cleaning
    'index keywords' : from scopus
    'author keywords' : from scopus
    'footnotes' : footnotes scraped from metadata panel.
    'raw' : raw text data scraped from JSTOR metadata panel.
    'references' : citations scraped from JSTOR metadata panel during data collection. 

### Plots of counts
Exploratory analysis of available data in scopus and datadumps.

In [ ]:
plt.rcParams["figure.figsize"] = (25, 10)
dta=Merged[(Merged['content_type']=='Article') & (Merged['year']>=1940)]['year'].value_counts()
plt.bar(dta.index,dta, label='Articles', alpha=0.4)
dta2=Merged[Merged['affiliations'].isnull()==False]['year'].value_counts()
plt.bar(dta2.index, dta2, label='Scopus coverage', alpha=0.4)
plt.title('Barchart of JPE Scopus coverage for affiliations overlaying counts of articles in JPE for each year between 1940 and 2016 (inclusive)')
plt.legend()

In [ ]:
plt.rcParams["figure.figsize"] = (25, 10)
dta=Merged[(Merged['content_type']=='Article') & (Merged['year']>=1940)]['year'].value_counts()
dta.index
plt.bar(dta.index, dta, alpha=0.4, label='Articles')
dta3=Merged[Merged['citations'].isnull()==False]['year'].value_counts()
plt.bar(dta3.index, dta3, alpha=0.4, label='Scopus coverage')
dta4=Merged[Merged['references'].isnull()==False]['year'].value_counts()
plt.bar(dta4.index, dta4, alpha=0.4, label='Datadump coverage')
plt.title('Barchart of JPE Scopus coverage for references and references scraped from JSTOR overlaying counts of articles in JPE for each year between 1940 and 2016 (inclusive)')
plt.legend()